In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install necessary libraries
!pip install -q unidecode fuzzywuzzy python-Levenshtein

# Import libraries
import re
import unidecode
import pandas as pd
from fuzzywuzzy import process, fuzz

# Mount Google Drive for accessing files
from google.colab import drive
drive.mount('/content/drive')

# File path to access datasets
main_path = '/content/drive/MyDrive/Master Paper/Data/'
file_path = f'{main_path}final_cleaned_results_67k.csv'
# Load the dataset
df = pd.read_csv(file_path, encoding='ISO-8859-1')

# Display dataset information
print("Initial dataset preview:")
print(df.head())
print(f"Dataset shape: {df.shape}")

# Lists of most known vehicle brands
known_car_brands = {'AC', 'AMC', 'Abarth', 'Acura', 'Alfa Romeo', 'Alpine', 'Aston Martin', 'Audi', 'Avanti', 'BMW', 'BYD', 'Bentley', 'Bugatti',
'Buick', 'Cadillac', 'Caterham', 'Changan', 'Chevrolet', 'Chrysler', 'Citroen', 'Cupra', 'DS', 'Dacia', 'Daewoo', 'Daihatsu',
'Datsun', 'DeLorean', 'Dodge', 'Eagle', 'FAW', 'Ferrari', 'Fiat', 'Fisker', 'Ford', 'Freightliner', 'GMC', 'Geely',
'Genesis', 'Geo', 'Great Wall', 'HUMMER', 'Honda', 'Hummer', 'Hyundai', 'Infiniti', 'Isuzu', 'Iveco', 'Jaguar', 'Jeep', 'Kia',
'Koenigsegg', 'Lada', 'Lamborghini', 'Lancia', 'Land Rover', 'Lexus', 'Lincoln', 'Lotus', 'MAN', 'MG', 'MINI', 'Mahindra',
'Maruti Suzuki', 'Maserati', 'Maxus', 'Maybach', 'Mazda', 'McLaren', 'Mercedes AMG', 'Mercedes-Benz', 'Mercury', 'Merkur',
'Mitsubishi', 'Morgan', 'Nissan', 'Oldsmobile', 'Opel', 'Pagani', 'Peel', 'Perodua', 'Peugeot', 'Pininfarina', 'Plymouth',
'Polestar', 'Pontiac', 'Porsche', 'Proton', 'RAM', 'Renault', 'Rimac', 'Rolls-Royce', 'Rover', 'SRT', 'Saab', 'Saturn', 'Scania',
'Scion', 'Seat', 'Skoda', 'SsangYong', 'Sterling', 'Subaru', 'Suzuki', 'TVR', 'Tata', 'Tesla', 'Toyota', 'Triumph', 'UAZ',
'Volkswagen', 'Volvo', 'Yugo', 'Zenvo', 'Zotye', 'smart'}

known_motorcycle_brands = {
    'AJP', 'Adiva', 'Aeon', 'Aprilia', 'Aspess Power', 'Axy', 'Azel', 'BMW', 'Bajaj', 'Benelli', 'Beta', 'Bimota',
    'Borile', 'Boss Hoss', 'Buell', 'Bultaco', 'CCM', 'CF Moto', 'CH Racing', 'CMC', 'CPI', 'CR&S', 'Cagiva',
    'Dado Motors', 'Daelim', 'Derbi', 'Ducati', 'E-Tropolis', 'E-max', 'Ecomission', 'Fantic Motor', 'Garelli',
    'Gas Gas', 'Generic', 'Ghezzi-Brian', 'GiMotori', 'GiPuma', 'Gilera', 'Green Mobility Italia', 'HDM', 'HM',
    'HP Power', 'Harley-Davidson', 'Headbanger', 'Honda', "Honda Dall'Ara", 'Hupper', 'Husaberg', 'Husqvarna',
    'Hyosung', 'Indian', 'Italjet', 'Jawa', 'KRC', 'KTM', 'Kawasaki', 'Kawasaki KL', 'Keeway', 'Kreidler', 'Kymco',
    'LML', 'Lambretta', 'Laverda', 'Leonart', 'Lingben', 'Linhai', 'MBK', 'MV Agusta', 'MZ', 'Magni', 'Maico',
    'Malaguti', 'Mash', 'Millepercento', 'Mondial', 'Montesa', 'Moto Bellini', 'Moto Guzzi', 'Moto Morini',
    'Moto Rumi', 'MotoBi', 'Motom', 'Motor Union', 'Nipponia', 'Norton', 'Nox', 'Ossa', 'Over', 'PGO', 'Paton',
    'Peda Motor', 'Peugeot', 'Piaggio', 'Polini', 'Quadro', 'Quantya', 'RedMoto Honda', 'Renault', 'Rieju',
    'Royal Enfield', 'SWM', 'Sachs', 'Scorpa', 'Sherco', 'Siamoto', 'Steed', 'Suzuki', 'Suzuki Valenti', 'Sym',
    'TGB', 'TM Racing', 'Terra Modena', 'Triumph', 'Ural', 'Vectrix', 'Vertemati', 'Victory', 'Vor', 'WT Motors',
    'Yamaha'
}


known_bus_brands = {
    'MAN', 'Mercedes Benz', 'Iveco', 'Scania', 'Volvo', 'Setra', 'Van Hool', 'Neoplan', 'Solaris', 'Mercedes-Benz',
    'Alexander Dennis', 'BYD', 'Blue Bird', 'New Flyer', 'Gillig', 'Temsa', 'Otokar', 'King Long', 'Yutong',
    'MCI', 'Isuzu', 'Tata', 'Ashok Leyland', 'Marcopolo', 'Higer', 'Golden Dragon', 'VDL', 'Leyland'
}


known_truck_brands = {
    'Mercedes Benz', 'Volvo', 'Scania', 'DAF', 'MAN', 'Iveco', 'Renault Trucks', 'Ford Trucks', 'Peterbilt',
    'Kenworth', 'Mack', 'Freightliner', 'Western Star', 'International', 'Tatra', 'Kamaz', 'Hino', 'Isuzu',
    'UD Trucks', 'Foton', 'Sinotruk', 'FAW', 'Dongfeng', 'Hyundai', 'Mahindra', 'Ashok Leyland', 'Tata', 'Dennis Eagle'
}

known_moped_brands = {
    'Piaggio', 'Vespa', 'Kymco', 'SYM', 'Peugeot', 'Yamaha', 'Honda', 'Derbi', 'Aprilia', 'Rieju',
    'Tomos', 'Puch', 'Gilera', 'Beta', 'Znen', 'TGB', 'Malaguti', 'Baotian', 'Zhongneng', 'Keeway', 'Benelli',
    'Adly', 'CPI', 'Generic', 'Lambretta', 'Motobecane', 'Sachs', 'Mash'
}


known_special_vehicle_brands = {
    'Mercedes Benz', 'MAN', 'Scania', 'Volvo', 'Ford', 'Iveco', 'Renault', 'DAF', 'Unimog', 'Tatra', 'Magirus',
    'Terberg', 'BMC', 'Rosenbauer', 'SISU', 'Oshkosh', 'Bremach', 'Kamaz', 'BelAZ', 'MTU', 'Hägglunds',
    'Foton', 'Dongfeng', 'Sinotruk', 'FAW', 'Mack Defense'
}


# Merge all known brands into one set
all_known_brands = known_car_brands.union(
    known_motorcycle_brands,
    known_bus_brands,
    known_truck_brands,
    known_moped_brands,
    known_special_vehicle_brands,
)

# Function to clean brand names
def clean_brand_name(name):
    """
    Normalize and clean brand names by removing extra information such as unnecessary characters or words.

    :param name: The brand name as a string to be cleaned.
    :return: A cleaned and normalized brand name as a string.
    """
    if not isinstance(name, str):
        return name
    name = unidecode.unidecode(name)  # Normalize characters
    name = re.split(r'[/\\:]', name)[0].strip()  # Remove extra info after certain characters
    if name.count(' ') >= 2:
        name = ' '.join(name.split()[:2])  # Keep only the first two words
    return name

# Function to match brand names using fuzzy matching
def match_brand_name(name, known_brands, short_threshold=80, long_threshold=85, min_length=6):
    """
    Match a brand name against known brands using fuzzy matching with stricter thresholds for longer names.

    :param name: The brand name to match.
    :param known_brands: A set of known brand names.
    :param short_threshold: Matching threshold for short brand names (default: 80).
    :param long_threshold: Matching threshold for long brand names (default: 85).
    :param min_length: Minimum length to consider a brand name as "long" (default: 6).
    :return: A tuple (matched_name, needs_manual_review, score).
    """
    if not isinstance(name, str) or len(name.strip()) == 0:
        return None, True, 0  # Invalid name, flag for manual review

    match, score = process.extractOne(name, known_brands, scorer=fuzz.token_set_ratio)
    if len(name) >= min_length:
        if score >= long_threshold:
            return match, False, score
        return name, True, score
    else:
        if score >= short_threshold:
            return match, False, score
        return name, True, score

# Apply cleaning to 'cleaned_marke' column
df['cleaned_marke'] = df['cleaned_marke'].apply(clean_brand_name)

# Extract unique cleaned brands
unique_cleaned_marke = df['cleaned_marke'].unique()
unique_brands_df = pd.DataFrame(unique_cleaned_marke, columns=['cleaned_marke'])

# Apply fuzzy matching to unique brands
unique_brands_df['matched_brand'], unique_brands_df['manual_flag'], unique_brands_df['score'] = zip(
    *unique_brands_df['cleaned_marke'].apply(lambda x: match_brand_name(x, all_known_brands))
)

# Merge matched brands back into the main dataset
df = df.merge(unique_brands_df[['cleaned_marke', 'matched_brand', 'manual_flag', 'score']], on='cleaned_marke', how='left')

# Update 'cleaned_marke' where the match is confident
df.loc[df['manual_flag'] == False, 'cleaned_marke'] = df['matched_brand']

# Keep necessary columns
df = df[['marke', 'modelis', 'cleaned_marke', 'cleaned_modelis']]

# Load car makers and models dataset
car_makers_models = pd.read_csv(f'{main_path}unique_car_makers_models_list.csv')
car_makers_models.rename(columns={'brand': 'cleaned_marke', 'model': 'modelis'}, inplace=True)

moto_makers_models = pd.read_csv(file_path + 'moto_makers_models_fuel_consumptions_cleaned.csv')

# Function to match model names within a specific brand
def match_model_name(model, cleaned_brand, car_makers_models):
    """
    Match model names within a specific brand using fuzzy matching.

    :param model: Model name to match.
    :param cleaned_brand: The corresponding brand name.
    :param car_makers_models: DataFrame containing known models for each brand.
    :return: A tuple (matched_model_name, needs_manual_review, score).
    """
    if not isinstance(model, str) or not isinstance(cleaned_brand, str):
        return None, True, 0  # Invalid data, flag for review

    brand_models = car_makers_models[car_makers_models['cleaned_marke'].str.lower() == cleaned_brand.lower()]
    if brand_models.empty:
        return None, True, 0  # No models found for the brand

    result = process.extractOne(model, brand_models['modelis'], scorer=fuzz.token_set_ratio)
    print(result)
    if result:
        matched_model_name, score = result
        if len(model) >= 6 and score >= 100:
            return matched_model_name, False, score
        if score >= 100:
            return matched_model_name, False, score
    return model, True, 0  # Flag for review

# Clean and match model names
df['modelis'] = df['modelis'].apply(clean_brand_name)
unique_models_df = df[['modelis', 'cleaned_marke']].drop_duplicates()
fuzzy_results = unique_models_df.apply(lambda row: match_model_name(row['modelis'], row['cleaned_marke'], car_makers_models), axis=1)
unique_models_df['matched_model'], unique_models_df['manual_flag'], unique_models_df['score'] = zip(*fuzzy_results)

# Merge matched models back into the main dataset
df = df.merge(unique_models_df[['modelis', 'matched_model', 'manual_flag', 'score']], on='modelis', how='left')

# Update 'cleaned_modelis' where the match is confident
df.loc[df['manual_flag'] == False, 'cleaned_modelis'] = df['matched_model']

# Save the cleaned dataset
df_cleaned = df[['marke', 'modelis', 'cleaned_marke', 'cleaned_modelis']].drop_duplicates()
#df_cleaned.to_csv(file_path + 'unique_names_regitra.csv', index=False)

# Display completion message
print("Cleaning process completed. Cleaned dataset saved to 'unique_names_regitra.csv'.")


In [ ]:
# Function to clean the brand name by normalizing and removing extra information
def clean_brand_name(name):
    """Clean brand name by normalizing and removing extra information."""
    if not isinstance(name, str):
        return name
    # Normalize characters
    name = unidecode.unidecode(name)
    # Remove any extra info after certain characters like slashes or second space if exists
    name = re.split(r'[/\\:]', name)[0].strip()
    return name

# Function to apply fuzzy matching for models based on cleaned brand (first filter by brand then match the model)
def match_model_name(model, cleaned_brand, car_makers_models):
    """Matches the model name for a specific cleaned brand using fuzzy matching within that brand."""
    # Ensure both model and cleaned_brand are strings, if not, return None
    if not isinstance(model, str) or not isinstance(cleaned_brand, str):
        return None, True, 0

    # First filter car_makers_models by the brand
    brand_models = car_makers_models[car_makers_models['cleaned_marke'].str.lower() == cleaned_brand.lower()]

    # If there are no models for the given brand, return None and flag it for manual review
    if brand_models.empty:
        return None, True, 0

    # Perform token_set_ratio fuzzy matching for better word-level similarity within the filtered models
    result = process.extractOne(model, brand_models['modelis'], scorer=fuzz.token_set_ratio)

    if result is None:
        return None, True, 0

    matched_model_name, score = result[0], result[1]

    # Set matching thresholds (similar to previous function)
    short_threshold = 100
    long_threshold = 100
    min_length = 6

    # Stricter matching for longer names
    if len(model) >= min_length:
        if score >= long_threshold:
            return matched_model_name, False, score  # High-confidence match, no manual review
        else:
            return model, True, score  # Flag for manual review
    else:
        if score >= short_threshold:
            return matched_model_name, False, score  # High-confidence match, no manual review
        else:
            return model, True, score  # Flag for manual review

# Clean the 'modelis' column before matching
df['modelis'] = df['modelis'].apply(clean_brand_name)

# Extract unique models and their corresponding brands
unique_models_df = df[['modelis', 'cleaned_marke']].drop_duplicates()

# Apply fuzzy matching for models within the respective brand using the new function
fuzzy_results = unique_models_df.apply(lambda row: match_model_name(row['modelis'], row['cleaned_marke'], car_makers_models), axis=1)

# Split the fuzzy results into separate columns
unique_models_df['matched_model'], unique_models_df['manual_flag'], unique_models_df['score'] = zip(*fuzzy_results)

# Remove duplicate rows before merging
unique_models_df = unique_models_df.drop_duplicates(subset=['modelis'])

# Map the matched models back to the main dataset (use 'left' or 'inner' based on the situation)
df = df.merge(unique_models_df[['modelis', 'matched_model','manual_flag','score']], on='modelis', how='left')

# Display flagged entries for manual inspection
flagged_entries = unique_models_df[unique_models_df['manual_flag']]

# Update 'cleaned_modelis' with 'matched_model' values if 'matched_model' is not null
# Tik false reikšmes iš 'manual_flag'
df.loc[df['manual_flag'] == False, 'cleaned_modelis'] = df['matched_model']

# Function to apply fuzzy matching for models based on cleaned brand (first filter by brand then match the model)
def match_model_name(model, cleaned_brand, moto_makers_models):
    """Matches the model name for a specific cleaned brand using fuzzy matching within that brand."""
    # Ensure both model and cleaned_brand are strings, if not, return None
    if not isinstance(model, str) or not isinstance(cleaned_brand, str):
        return None, True, 0

    # First filter moto_makers_models by the brand
    brand_models = moto_makers_models[moto_makers_models['cleaned_marke'].str.lower() == cleaned_brand.lower()]

    # If there are no models for the given brand, return None and flag it for manual review
    if brand_models.empty:
        return None, True, 0

    # Perform token_set_ratio fuzzy matching for better word-level similarity within the filtered models
    result = process.extractOne(model, brand_models['modelis'], scorer=fuzz.token_set_ratio)

    if result is None:
        return None, True, 0

    matched_model_name, score = result[0], result[1]

    # Set matching thresholds (similar to previous function)
    short_threshold = 100
    long_threshold = 100
    min_length = 6

    # Stricter matching for longer names
    if len(model) >= min_length:
        if score >= long_threshold:
            return matched_model_name, False, score  # High-confidence match, no manual review
        else:
            return model, True, score  # Flag for manual review
    else:
        if score >= short_threshold:
            return matched_model_name, False, score  # High-confidence match, no manual review
        else:
            return model, True, score  # Flag for manual review

# Clean the 'modelis' column before matching
df['modelis'] = df['modelis'].apply(clean_brand_name)

# Extract unique models and their corresponding brands
unique_models_df = df[['modelis', 'cleaned_marke']].drop_duplicates()

# Apply fuzzy matching for models within the respective brand using the new function
fuzzy_results = unique_models_df.apply(lambda row: match_model_name(row['modelis'], row['cleaned_marke'], moto_makers_models), axis=1)

# Split the fuzzy results into separate columns
unique_models_df['matched_model'], unique_models_df['manual_flag'], unique_models_df['score'] = zip(*fuzzy_results)

# Remove duplicate rows before merging
unique_models_df = unique_models_df.drop_duplicates(subset=['modelis'])

# Map the matched models back to the main dataset (use 'left' or 'inner' based on the situation)
df = df.merge(unique_models_df[['modelis', 'matched_model','manual_flag','score']], on='modelis', how='left')

# Display flagged entries for manual inspection
flagged_entries = unique_models_df[unique_models_df['manual_flag']]

# Update 'cleaned_modelis' with 'matched_model' values if 'matched_model' is not null
# Tik false reikšmes iš 'manual_flag'
df.loc[df['manual_flag'] == False, 'cleaned_modelis'] = df['matched_model']

